In [ ]:
%config Completer.use_jedi = False

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import time
from sklearn.cluster import DBSCAN

import src.fusion as fusion
import src.utils.clip_utils as clip_utils
import src.utils.las_utils as las_utils
from src.utils.labels import Labels
from src.region_growing import LabelConnectedComp

## Set-up

In [ ]:
# Data folder for the BGT fuser.
bgt_data_folder = '../datasets/bgt/'
bgt_data_file = '../datasets/bgt/bgt_points.csv'
tile_code = '2386_9702'
#tile_code = '2397_9705'

# Building fuser using BGT building footprint data.
bgt_point_fuser = fusion.BGTPointFuser(Labels.TREE, bgt_file=bgt_data_file)

In [ ]:
# Extract pole objects from tile.
bgt_points = bgt_point_fuser._filter_tile(tile_code)
trees = [[x, y] for (t, x, y) in bgt_points if t == 'boom']
lights = [[x, y] for (t, x, y) in bgt_points if t == 'lichtmast']
signs = [[x, y] for (t, x, y) in bgt_points if t == 'verkeersbord']

In [ ]:
# Load labeled and grown LAS file.
las_file = '../datasets/pointcloud/grown_' + tile_code + '.laz'
las = las_utils.read_las(las_file)

In [ ]:
# Remove ground and building points.
mask = (las.label != Labels.GROUND) & (las.label != Labels.BUILDING)
mask_ids = np.where(mask)[0]
points = np.vstack((las.x[mask], las.y[mask], las.z[mask])).T
n_points = len(points)

# Compute average ground elevation.
ground_mask = las.label == Labels.GROUND
avg_ground_height = np.mean(las.z[ground_mask])

## Clustering

In [ ]:
# Define a plane to search for seed points.
plane_height = 2.
plane_buffer = 0.25
seed_plane_mask = (((avg_ground_height + plane_height - plane_buffer) < points[:,2]) 
                   & (points[:,2] < (avg_ground_height + plane_height + plane_buffer)))

print(f'We have {np.count_nonzero(seed_plane_mask)} potential seed points')

In [ ]:
# Get <X, Y> of potential seed points.
points_xy = points[seed_plane_mask, 0:2]

In [ ]:
# Cluster the potential seed points.
clustering = DBSCAN(eps=0.05, min_samples=15, p=2).fit(points_xy)
#clustering = OPTICS(max_eps=0.1, min_samples=5, p=2, min_cluster_size=50).fit(points_xy)

# Remove noise points.
noise_mask = clustering.labels_ != -1

In [ ]:
# Optional: filter clusters by size.

# Get cluster labels and sizes.
cc_labels, counts = np.unique(clustering.labels_, return_counts=True)

# Only keep clusters with size between N_min and N_max.
N_min = 100
N_max = 5000
count_valid = np.where((counts >= N_min) & (counts <= N_max))

# Update noise mask.
noise_mask = noise_mask & [l in set(cc_labels[count_valid]) for l in clustering.labels_]

In [ ]:
# Filter points based on noise mask.
points_xy_filter = points_xy[noise_mask,:]
print(f'We have {np.count_nonzero(noise_mask)} seed points left after noise filtering.')

In [ ]:
# Visualize the results.
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.patches as patches

bbox = las_utils.get_bbox_from_tile_code(tile_code)

fig, ax = plt.subplots(1)
ax.scatter(points_xy_filter[:,0], points_xy_filter[:,1], c=clustering.labels_[noise_mask], marker='.')

for tree in trees:
    plt.scatter(tree[0], tree[1], c='green', marker='x')

for light in lights:
    plt.scatter(light[0], light[1], c='purple', marker='x')
    
for sign in signs:
    plt.scatter(sign[0], sign[1], c='red', marker='x')

((x_min, y_max), (x_max, y_min)) = bbox
box = patches.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, linewidth=1, linestyle='--', edgecolor='grey', fill=False)
ax.add_patch(box)

ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.axis('equal')
plt.show()

## Cluster matching

In [ ]:
# Create a list of cluster centers (x,y) and radius r.
c_xyr_list = []
for cl in cc_labels[count_valid]:
    c_mask = clustering.labels_ == cl
    (cx, cy) = np.mean(points_xy[c_mask,:], axis=0)
    cr = np.max(np.max(points_xy[c_mask,:], axis=0) - np.min(points_xy[c_mask,:], axis=0)) / 2
    c_xyr_list.append([cx, cy, cr])
c_xyr_list = np.array(c_xyr_list)

In [ ]:
# Match BGT point objects to nearby clusters.
max_dist = 1.5

tree_points = []
light_points = []
sign_points = []

for tree in trees:
    dist = [np.linalg.norm(np.array(tree) - np.array([cxy])) for cxy in c_xyr_list[:,0:2]]
    if np.min(dist) < max_dist:
        tree_points.append(c_xyr_list[np.argmin(dist)])

for light in lights:
    dist = [np.linalg.norm(np.array(light) - np.array([cxy])) for cxy in c_xyr_list[:,0:2]]
    if np.min(dist) < max_dist:
        light_points.append(c_xyr_list[np.argmin(dist)])

for sign in signs:
    dist = [np.linalg.norm(np.array(sign) - np.array([cxy])) for cxy in c_xyr_list[:,0:2]]
    if np.min(dist) < max_dist:
        sign_points.append(c_xyr_list[np.argmin(dist)])

In [ ]:
# Visualize the resulting match.

fig, ax = plt.subplots(1)
ax.scatter(points_xy_filter[:,0], points_xy_filter[:,1], c='lightgrey', marker='.')

for tree in tree_points:
    circle = patches.Circle((tree[0], tree[1]), tree[2], linewidth=2, linestyle='-', edgecolor='green', fill=False)
    ax.add_patch(circle)

for light in light_points:
    circle = patches.Circle((light[0], light[1]), light[2], linewidth=2, linestyle='-', edgecolor='purple', fill=False)
    ax.add_patch(circle)

for sign in sign_points:
    circle = patches.Circle((sign[0], sign[1]), sign[2], linewidth=2, linestyle='-', edgecolor='red', fill=False)
    ax.add_patch(circle)

((x_min, y_max), (x_max, y_min)) = bbox
box = patches.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, linewidth=1, linestyle='--', edgecolor='grey', fill=False)
ax.add_patch(box)

ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.axis('equal')
plt.show()

## Labelling

In [ ]:
# Optional: re-load labeled and grown LAS file.
# E.g. to try out different settings.
las_file = '../datasets/pointcloud/grown_' + tile_code + '.laz'
las = las_utils.read_las(las_file)

In [ ]:
# Do the actual labelling.
# Cluster radius tends to be too small (although sometimes it isn't) so we multiply.
r_mult = 2

tree_mask = np.zeros((n_points,), dtype=bool)
light_mask = np.zeros((n_points,), dtype=bool)
sign_mask = np.zeros((n_points,), dtype=bool)

for tree in tree_points:
    clip_mask = clip_utils.circle_clip(points, tree[0:2], r_mult*tree[2])
    tree_mask = tree_mask | clip_mask

for light in light_points:
    clip_mask = clip_utils.circle_clip(points, light[0:2], r_mult*light[2])
    light_mask = light_mask | clip_mask

for sign in sign_points:
    clip_mask = clip_utils.circle_clip(points, sign[0:2], r_mult*sign[2])
    sign_mask = sign_mask | clip_mask

In [ ]:
# Create labels.
labels = las.label
labels[mask_ids[tree_mask]] = Labels.TREE
labels[mask_ids[light_mask]] = Labels.STREET_LIGHT
labels[mask_ids[sign_mask]] = Labels.TRAFFIC_SIGN

In [ ]:
# Cluster based region growing.
# Not for trees yet.
lcc_light = LabelConnectedComp(Labels.STREET_LIGHT, exclude_labels=(Labels.TREE, Labels.TRAFFIC_SIGN),
                               octree_level=9, min_component_size=100, threshold=0.4)
lcc_sign = LabelConnectedComp(Labels.TRAFFIC_SIGN, exclude_labels=(Labels.TREE, Labels.STREET_LIGHT),
                              octree_level=9, min_component_size=100, threshold=0.4)

lcc_light_mask = lcc_light.get_label_mask(points=points, las_labels=las.label[mask])
lcc_sign_mask = lcc_sign.get_label_mask(points=points, las_labels=las.label[mask])

# Update labels.
labels[mask_ids[lcc_light_mask]] = lcc_light.get_label()
labels[mask_ids[lcc_sign_mask]] = lcc_sign.get_label()

In [ ]:
# Save the result.
out_file = '../datasets/pointcloud/poles_' + tile_code + '.laz'
las_utils.label_and_save_las(las, labels, out_file)